In [1]:
from ultralytics import YOLO
import os
import ipywidgets as widgets

In [2]:
model = YOLO('./best.pt')
names = model.names

In [3]:
results = model(['./letters/a/a_1.jpg'])



0: 640x384 1 A, 105.2ms
Speed: 3.5ms preprocess, 105.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)


In [4]:
# Cargar letras.
letters = {}
for folder in os.listdir('./letters'):
    letter_images = os.listdir('./letters/' + folder)
    if len(letter_images) > 0:
        letters[folder] = []
    for file in os.listdir('./letters/' + folder):
        letters[folder].append(file)

In [5]:
# Dropdowns.

letter_dropdown = widgets.Dropdown(options=letters.keys())
image_selector = widgets.Dropdown()
def on_letter_dropdown_change(change):
    if change['name'] == 'value' and change['new'] is not None:
        image_selector.options = letters[change['new']]
letter_dropdown.observe(on_letter_dropdown_change)
on_letter_dropdown_change({'name' : 'value', 'new': letter_dropdown.value}) # Inicializa el segundo dropdown.

# Label de predicción.

label_pred = widgets.Label(value='Predicción: ')

# Carga el widget de imagen.
image_widget = widgets.Image(format='jpg', width=300, height=400)
def on_image_selector_change(change):
    if change['name'] == 'value' and change['new'] is not None:
        file_path = './letters/' + letter_dropdown.value + '/' + change['new']
        file = open(file_path, "rb")
        image_widget.value = file.read()
        file.close()
        prediction = model.predict(file_path, verbose=False)[0]
        if len(prediction.boxes.cls) == 0:
            label_pred.value = 'Predicción: No se encontró ninguna letra.'
        else:
            label_pred.value = 'Predicción: ' + names[int(prediction.boxes.cls[0])]
image_selector.observe(on_image_selector_change)
on_image_selector_change({'name' : 'value', 'new': image_selector.value}) # Inicializa la imagen.
# Mostrar dropdowns e imagen.
display(widgets.HBox([letter_dropdown, image_selector]))
display(widgets.HBox([image_widget, label_pred]))

: 